**Enoncé:**

On va s'interesser aux données entreprise disponible sur le site de Reuters.
Besoin métier: Analyser les performances financières des sociétés cotées pour décider d'une stratégie d'investissement.

Je vous demande donc de récupérer les infos suivantes :
* les ventes au quartier à fin décembre 2018
* le prix de l'action et son % de changement au moment du crawling
* le % Shares Owned des investisseurs institutionels
* le dividend yield de la company, le secteur et de l'industrie

pour les sociétés suivantes : Aribus, LVMH et Danone.

----------

In [7]:
# coding: utf-8
import requests
import unittest
from bs4 import BeautifulSoup

### Récupérer la page

In [55]:
def _handle_request_result_and_build_soup(request_result):
    if request_result.status_code == 200:
        html_doc =  request_result.text
        soup = BeautifulSoup(html_doc,"html.parser")
    return soup

### Les ventes au quarter à fin décembre 2018

In [188]:
def get_sales_quarter(soup):
    return soup.find_all('td', attrs={'class': 'data'})[1].get_text()

### Le prix de l'action et son % de changement au moment du crawling

In [236]:
def get_share_price(soup):
    section = soup.find('div', attrs={'class': 'sectionQuote nasdaqChange'})
    share_price = section.find('span', attrs={'style': 'font-size: 23px;'}).get_text().strip()
    return share_price

In [237]:
def get_pct_chg(soup):
    section = soup.find('div', attrs={'class': 'sectionQuote priceChange'})
    pct_chg = section.find('span', attrs={'class': 'valueContentPercent'}).get_text()
    return pct_chg

### Shares Owned des investisseurs institutionels

In [308]:
def get_shares_owned_inst_holders(soup):
    section = soup.find('div', class_="column2 gridPanel grid4")
    module = section.find_all('div', class_={'module'})[3]
    shares = module.find('td', class_='data').get_text()
    return shares

### Dividend yield de la compagnie, du secteur et de l'industrie

In [340]:
def get_divs_yield(soup):
    section = soup.find('div', class_="column1 gridPanel grid8")
    module = section.find_all('div', class_={'module'})[3]
    div = module.find_all('td', class_='data')
    div_company = div[0].get_text()
    div_industry = div[1].get_text()
    div_sector = div[2].get_text()
    return div_company, div_industry, div_sector

### Affichage du résultat

In [351]:
import pandas as pd
df = pd.DataFrame(index = companies, columns = ['Ventes',
                                                "Prix de l'action",
                                                "% du changement de l'action",
                                                "Shares Owned des investisseurs institutionels",
                                                "Dividendes"])
def print_result(companies):
    for c in companies:
        res = requests.get("https://www.reuters.com/finance/stocks/financial-highlights/" + c)
        soup = _handle_request_result_and_build_soup(res)
        sales_mean = get_sales_quarter(soup)
        share_price = get_share_price(soup)
        pct_chg = get_pct_chg(soup)
        shares = get_shares_owned_inst_holders(soup)
        dividends = get_divs_yield(soup)
        df.loc[c] = pd.Series({'Ventes': sales_mean,
                               "Prix de l'action": share_price,
                               "% du changement de l'action": pct_chg,
                               "Shares Owned des investisseurs institutionels": shares,
                                "Dividendes": dividends})
    return df

In [352]:
print_result(['AIR.PA', 'DANO.PA', 'LVMH.PA'])

,Ventes,Prix de l'action,% du changement de l'action,Shares Owned des investisseurs institutionels,Dividendes
AIR.PA,"23,493.00",101.36,(-1.11%),43.53%,"(1.45, 1.34, 1.64)"
DANO.PA,"6,072.60",65.38,(-0.40%),50.60%,"(2.90, 2.78, 2.48)"
LVMH.PA,"13,667.70",269.00,(-5.85%),20.57%,"(1.92, 1.70, 2.60)"
